# Part 1
# Data Discovery: Join Path Discovery

For the following data discovery lab, your objective is to find plausible joins between various tables in the data lake. Data lakes typically contain a collection of tables without any schema or meta-data (i.e., we have no idea how tables in the data lake are conenct to each other, unline in a database schema where we have Primary Key - Foreign Key relationships). That is, you will have to find join paths for the following questions below. A join path in data discovery is a query used to combine data from multiple tables based on common attributes or keys, employing suitable join metrics (i.e., column similairty). You need to determine which tables should be connected and identify the columns that will serve as the basis for these joins. Justify your choices by using the similarity metrics (e.g, edit distance, jaccard similarity, TFIDF with cosine similarity) depending on the nature of the data, providing justification to support why these columns are suitable for joining the tables. Your code should be properly commented describing each step.

### Q1.(10 pts) For movie title: 'Toy Story 3', get Popularity, Film certificate, Gross amount, Num_critic_for_reviews and Num_user_for_review.


In [1]:
# code
# Q1.

'''
In question 1, I followed certain steps to get the output:
1.) I have identified the movie 'Toy Story 3' values required in two csv files 'submission (3)' and 'facebook', I converted them into dataframes.
2.) In facebook_df, I extracted the rows with budget = 200000000 which is the Toy Story's budget and extracted them into a new dataframe
3.) I used jaccard similarity for comparing the string5 column in submission_df with genres column in new_df
'''

import pandas as pd
from sklearn.metrics import jaccard_score

submission_df = pd.read_csv('data/submission (3).csv')
facebook_df = pd.read_csv('data/facebook.csv')

facebook_df = facebook_df.dropna(subset=['budget'])
facebook_df['budget'] = facebook_df['budget'].astype(int)

ts3 = submission_df[submission_df['string1'] == 'Toy Story 3']

ts3_budget = ts3['budget'].iloc[0]

new_df = facebook_df[facebook_df['budget'] == ts3_budget]

new_df['genres'] = new_df['genres'].str.replace('|', ', ')

string5_set = set(submission_df['string5'].dropna().unique())
genres_set = set(new_df['genres'].dropna().unique())

intersection = len(string5_set.intersection(genres_set))
union = len(string5_set.union(genres_set))

jaccard_similarity = intersection / union

# Jaccard Similairty of the compared columns
print("\nJaccard Similarity between 'string5' and 'genres':", jaccard_similarity)


ts3_str5 = submission_df.loc[submission_df['string1'] == 'Toy Story 3', 'string5'].iloc[0]

dir_genres = new_df.loc[new_df['director_name'] == 'Lee Unkrich', 'genres'].iloc[0]

ts3_str5_set = set(ts3_str5.split(', '))
dir_genres_set = set(dir_genres.split(', '))

ts3_intersection = len(ts3_str5_set.intersection(dir_genres_set))
ts3_union = len(ts3_str5_set.union(dir_genres_set))

jaccard_similarity = ts3_intersection / ts3_union

# Jaccard Similarity of the genres
print("\nJaccard Similarity between 'Toy Story 3' string5 in submission_df and Lee Unkrich's genres in new_df:", jaccard_similarity)



Jaccard Similarity between 'string5' and 'genres': 0.0005494936808226706

Jaccard Similarity between 'Toy Story 3' string5 in submission_df and Lee Unkrich's genres in new_df: 0.6


/tmp/ipykernel_6107/3284574026.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['genres'] = new_df['genres'].str.replace('|', ', ')


In [2]:
# Q1.

'''
4.) To match the record, I have manually taken the director name of Toy Story 3 and compared the genres of that movie in both the dataframes
5.) As the similarity score is 0.00055 for columns and 0.6 for Toy Story 3's genres, I have extracted the required attributes.
ts3_popularity = submission_df.loc[submission_df['string1'] == 'Toy Story 3', 'popularity'].iloc[0]
'''

ts3_popularity = submission_df.loc[submission_df['string1'] == 'Toy Story 3', 'popularity'].iloc[0]

# I have manually checked the director name and used it to extract attributes. (Lee Unkrich as director_name)
director_info = new_df.loc[new_df['director_name'] == 'Lee Unkrich', ['content_rating', 'gross', 'num_critic_for_reviews',
                                                                      'num_user_for_reviews']].iloc[0]


print('Toy Story 3 attributes required: \n')


ts3_values_dict = {
    'Popularity': ts3_popularity,
    'Content Rating': director_info['content_rating'],
    'Gross': director_info['gross'],
    'Num Critic for Reviews': director_info['num_critic_for_reviews'],
    'Num User for Reviews': director_info['num_user_for_reviews']
}

for key, value in ts3_values_dict.items():
    print(key + ':', value)

Toy Story 3 attributes required: 

Popularity: 69.145
Content Rating: G
Gross: 414984497.0
Num Critic for Reviews: 453.0
Num User for Reviews: 733.0


### Q2.(10 pts) Find out the combined total of Oscar awards won by the actors throughout their careers, who starred in 'Avengers: Endgame'



In [3]:
# code
# Q2.

'''
In question 2, I followed certain steps to get the output:
1.) I have identified that for the movie 'Avengers: Endgame', the cast members names can be found using in broadcast csv,
    the top100 csv gives the actors with oscar wins, but the column has slight variations within the names.
2.) First I found the jaccard similarity for the Performr column and Nmae column but because of the variation,
    Jaccard similarity didn't seem logical here.
3.) I have created a new_df which takes only the actors who have performed in 'Avengers: Endgame'
'''

import pandas as pd

broadcast_df = pd.read_csv('data/broadcast.csv')
top100_df = pd.read_csv('data/top100.csv')

performer_set = set(broadcast_df['Performr'].dropna().unique())
name_set = set(top100_df['Nmae'].dropna().unique())

intersection = len(performer_set.intersection(name_set))
union = len(performer_set.union(name_set))
jaccard_similarity = intersection / union

print("Jaccard Similarity between Performer and Actor Name:", jaccard_similarity)


new_df = broadcast_df[broadcast_df['feature'] == 'Avengers: Endgame']

print("Endgame DataFrame:")
print(new_df.shape)
new_df

#new_df['Performr']

Jaccard Similarity between Performer and Actor Name: 0.0015463917525773195
Endgame DataFrame:
(68, 7)


,Unnamed: 0,Performr,ActorID,feature,Year,Votes,Rating
5695,5695,Michael Douglas,nm0000140,Avengers: Endgame,2019,849079,8.4
7108,7108,Samuel L. Jackson,nm0000168,Avengers: Endgame,2019,849079,8.4
8634,8634,Michelle Pfeiffer,nm0000201,Avengers: Endgame,2019,849079,8.4
8731,8731,Natalie Portman,nm0000204,Avengers: Endgame,2019,849079,8.4
12143,12143,Angela Bassett,nm0000291,Avengers: Endgame,2019,849079,8.4
...,...,...,...,...,...,...,...
187846,187846,Tom Holland,nm4043618,Avengers: Endgame,2019,849079,8.4
189237,189237,Carrie Coon,nm4689420,Avengers: Endgame,2019,849079,8.4
191063,191063,Winston Duke,nm6328300,Avengers: Endgame,2019,849079,8.4
191722,191722,Jacob Batalon,nm8188622,Avengers: Endgame,2019,849079,8.4


In [4]:
# Q2.

'''
4.) Instead of Jaccard Similarity, I have used editdistance as it deals with character variations. 
    I have used it on Performr and Nmae column and kept the threshold value > 0.6 for the similarity score.
5.) Hence, it gave me the total number of Oscar wins for the cast to be 10.
'''

import editdistance

def calculate_similarity(string1, string2):

    distance = editdistance.eval(string1.lower(), string2.lower())

    max_length = max(len(string1), len(string2))
    similarity_score = 1 - (distance / max_length)
    
    return similarity_score

count = 0

for index, row in new_df.iterrows():
    performer_name = row['Performr']

    for name in top100_df['Nmae']:
        similarity_score = calculate_similarity(performer_name, name)
        if similarity_score > 0.6:
            matching_rows = top100_df[top100_df['Nmae'] == name]

            for index, match_row in matching_rows.iterrows():
                oscar_count = match_row['Oscars']
                count = count + oscar_count

print("Total Oscars won by performers in 'Avengers: Endgame':", count)


Total Oscars won by performers in 'Avengers: Endgame': 10


### Q3.(10 pts) Tell the net profit of movies with 'Comedy' genre and which of them have won golden globe and in which category?



In [5]:
# code
# Q3. 

'''
1.) In question 3, I have seen that repo.csv contains the information about Golden Globe Awards based on genre. 
    As the candidate column is similar to the string1 of submissions (3).csv, I used them.
2.) To filter out movies with only genre as Comedy, I have extracted the string5 in submissions_df and created a new df (comedy_movies_df)
3.) In order to get the net_profit of these movies I have used net_profit = revenue - budget
'''

from fuzzywuzzy import fuzz

submission_df = pd.read_csv('data/submission (3).csv')
repo_df = pd.read_csv('data/repo.csv')

comedy_movies_df = submission_df[submission_df['string5']=='Comedy']

print(comedy_movies_df.shape)

comedy_movies_df['net_profit'] = comedy_movies_df['revenue'] - comedy_movies_df['budget']
comedy_pos_df=comedy_movies_df[comedy_movies_df['net_profit']>0].copy()

comedy_pos_df[['string1','revenue','budget','string5','net_profit']].head()

(56042, 15)


/tmp/ipykernel_6107/3578832672.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comedy_movies_df['net_profit'] = comedy_movies_df['revenue'] - comedy_movies_df['budget']


,string1,revenue,budget,string5,net_profit
95,The Hangover,469310836,35000000,Comedy,434310836
286,The Hangover Part II,586764305,80000000,Comedy,506764305
393,The Hangover Part III,362000072,103000000,Comedy,259000072
406,Mean Girls,130126277,17000000,Comedy,113126277
546,Superbad,170812526,20000000,Comedy,150812526


In [6]:
# Q3.

'''
4.) I created an empty list to check the string matches and category, looped them through string1 of comedy_movies_df  
5.) If the similarity score used is above the threshold, the list appends the data 
    (string1, candidate matched, category and if it has golden globe)
6.) I got a total of 49 records based on my similarity scores
'''

match_list = []
award_wins_df = repo_df[(repo_df['golden globe win'] == True)].copy()

for i, row in comedy_pos_df.iterrows():
    s = row['string1']
    string_match = ''
    val=0
    category = ''
    for i1, row1 in award_wins_df.iterrows():
        similarity = fuzz.ratio(s.lower(), row1['candidate'].lower())
        if similarity > val and similarity > 70:
            val = similarity
            string_match = row1['candidate']
            category = row1['category']
            
    if val > 70:
        match_list.append((s, string_match, category, True))
        
print(len(match_list))

49


In [7]:
# Q3.

'''
7.) I used a dataframe to print out the values in list clearly, which movie won golden globe in which category 
'''

win_category_df = pd.DataFrame(match_list)

win_category_df.rename(columns={0: 'string1', 1: 'candidate', 2: 'category',  3: 'golden globe win'}, inplace=True)
print(win_category_df.shape)
win_category_df.head()

(49, 4)


,string1,candidate,category,golden globe win
0,Borat: Cultural Learnings of America for Make ...,Borat: Culturael Learnings Of America For Make...,Best Performance by an Actor in a Motion Pictu...,True
1,American Reunion,American Beaut,Best Screenplay - Motion Picture,True
2,Life of Brian,Life eof Pi,Best Original Score - Motion Picture,True
3,Office Space,"Office, aThe",Best Performance by an Actor in a Television S...,True
4,This Is 40,eThis Is Us,Best Performance by an Actor In A Television S...,True


### Q4.(10 pts) Extract NULL movie titles from homepage URLs in the TMDB_null_title dataset and find their corresponding original titles using another dataset. Compute the similarity score using most appropriate similarity metric for each title pair, adding this as a new column in the dataframe. Also, display the sum of the score.

In [8]:
# code
# Q4.

'''
1.) In Q4, I used two datasets null_title and submission (3) to extract the movie name and then compare it with the string1 in submission (3).
2.) As given in the tutorial, I have used regex to extract the movie title:
    -> First, I have replaced the unnecessary parts of url with null character
    -> Then extracted the end part of the url which in most cases is the movie name, unless it is a domain
    -> For domains, I have extracted as a whole word, then eliminated the hyphens in movie names
    -> Capitalized the names which makes it similar to movie names in string1
    -> Applied it to the homepage column and used it as a new column
'''

import pandas as pd
null_title_df = pd.read_csv("data/null_title.csv")
submission_df = pd.read_csv("data/submission (3).csv")

import re


def movie_title(url):
    url = url.replace('http://', '').replace('https://', '').replace('www.', '').strip('/')
    split_parts = url.split('/')

    if len(split_parts) == 1:
        req_title = split_parts[0]
    else:
        req_title = split_parts[-1]

    name = re.sub(r'\.[a-zA-Z0-9]+$', '', req_title)

    name = re.sub(r'[-_]', ' ', name)

    name = name.split('.')[0]
    
    words = name.split()

    cap_words = []

    for word in words:
        cap_word = word.capitalize()
        cap_words.append(cap_word)

        name = ' '.join(cap_words)
        
    name = name.replace('movie','')
    return name

null_title_df['Movie Name'] = null_title_df['homepage'].apply(movie_title)

null_title_df.head()

,title,status,homepage,original_language,release_date,tagline,Movie Name
0,NaN,Released,https://www.warnerbros.com/movies/inception,en,2010-07-15,YOUR MIND IS THE SCENE OF THE CRIME. - 2010-07-15,Inception
1,NaN,Released,http://www.interstellarmovie.net/,en,2014-11-05,MANKIND WAS BORN ON EARTH. IT WAS NEVER MEANT ...,Interstellar
2,NaN,Released,https://www.warnerbros.com/movies/dark-knight/,en,2008-07-16,WELCOME TO A WORLD WITHOUT RULES. - 2008-07-16,Dark Knight
3,NaN,Released,https://www.avatar.com/movies/avatar,en,2009-12-15,ENTER THE WORLD OF PANDORA. - 2009-12-15,Avatar
4,NaN,Released,https://www.marvel.com/movies/the-avengers,en,2012-04-25,SOME ASSEMBLY REQUIRED. - 2012-04-25,The Avengers


In [9]:
# Q4.

'''
3.) Here I have taken a sample data for comparison of the files. I have then takrn a new list to record similarity scores 
    and the movie title list to check how similar are the titles.
4.) Then I have run a loop to compare each title with the ones in Movie Name column 
    and then used Jaccard Similarity to check the similarity scores.
5.) I added the columns Similar Scores and Similar Title to the null_title_df and finally calculated the sum of these scores.
'''

def jaccard_similarity(str1, str2):
    set1 = set(str1.lower().split())
    set2 = set(str2.lower().split())
    return len(set1.intersection(set2)) / len(set1.union(set2))
    
sample_comparison = submission_df['string1'].iloc[:35000]

sim_scores = []
titles = []

for i in null_title_df['Movie Name']:
    val = 0
    updated_title = ''
    for title in sample_comparison:
        title = str(title)
        score = jaccard_similarity(i, title)
        if score > val:
            val = score
            updated_title = title
    titles.append(updated_title)
    sim_scores.append(val)

null_title_df['Similarity Score'] = sim_scores
null_title_df['Similar Title'] = titles

for index, row in null_title_df.iterrows():
    if pd.isnull(row['Similar Title']):
        null_title_df.drop(row, inplace=True)

In [10]:
null_title_df.iloc[30:41]


,title,status,homepage,original_language,release_date,tagline,Movie Name,Similarity Score,Similar Title
30,NaN,Released,http://www.spidermanhomecoming.com,en,2017-07-05,HOMEWORK CAN WAIT. THE CITY CAN'T. - 2017-07-05,Spidermanhomecoming,0.000000,
31,NaN,Released,https://www.warnerbros.com/movies/harry-potter...,en,2002-11-13,HOGWARTS IS BACK IN SESSION. - 2002-11-13,Harry Potter And Chamber Secrets,0.714286,Harry Potter and the Chamber of Secrets
32,NaN,Released,http://marvel.com/movies/movie/221/guardians_o...,en,2017-04-19,OBVIOUSLY. - 2017-04-19,Guardians Of The Galaxy Vol 2,0.714286,Guardians of the Galaxy Vol. 2
33,NaN,Released,http://www.lordoftherings.net/,en,2002-12-18,A NEW POWER IS RISING. - 2002-12-18,Lordoftherings,0.000000,
34,NaN,Released,https://www.marvel.com/movies/captain-america-...,en,2011-07-22,WHEN PATRIOTS BECOME HEROES - 2011-07-22,Captain America The First Avenger,0.666667,Captain America: The First Avenger
35,NaN,Released,http://www.suicidesquad.com/,en,2016-08-03,WORST. HEROES. EVER. - 2016-08-03,Suicidesquad,0.000000,
36,NaN,Released,https://www.warnerbros.com/movies/harry-potter...,en,2004-05-31,SOMETHING WICKED THIS WAY COMES. - 2004-05-31,Harry Potter And Prisoner Azkaban,0.714286,Harry Potter and the Prisoner of Azkaban
37,NaN,Released,http://thor.marvel.com/,en,2011-04-21,TWO WORLDS. ONE HERO. - 2011-04-21,Thor,1.000000,Thor
38,NaN,Released,https://www.marvel.com/movies/iron-man-2,en,2010-04-28,"IT'S NOT THE ARMOR THAT MAKES THE HERO, BUT TH...",Iron Man 2,1.000000,Iron Man 2
39,NaN,Released,https://www.warnerbros.com/movies/batman-begins/,en,2005-06-10,EVIL FEARS THE KNIGHT. - 2005-06-10,Batman Begins,1.000000,Batman Begins


In [11]:
# Q4.
sum_of_scores = sum(sim_scores)

print('The sum of all similarity scores between Movie Name and Similar Title: ', sum_of_scores)

The sum of all similarity scores between Movie Name and Similar Title:  1142.5963980463973
